In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
## importing stuff
import os
import pickle
from sys import path as syspath
syspath.append(os.path.expanduser("~/srdjan_functs/"))
import javabridge
from bioformats import JARS as bfJARS
javabridge.start_vm(class_path=bfJARS, max_heap_size="20G")
import numpy as np
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2
# %aimport islets

from IPython.display import display
import pandas as pd

from islets.Recording import Recording, saveMovie, parse_leica
from islets.numeric import power_spectrum, rebin
from islets.utils import show_movie, saveRois
# from islets.Regions1 import getPeak2BoundaryDF, getGraph_of_ROIs_to_Merge, mergeBasedOnGraph

# from caiman import movie as cmovie

from matplotlib.colors import LogNorm

from islets.numeric import sosFilter

In [3]:
from islets.Recording import Recording, saveMovie, parse_leica

In [4]:
# %%capture
# from ExpLorer_5 import import_data

In [5]:
# status = import_data("/data/Sandra/2019/2019_08_13/","29b")

In [6]:
folder = "/data/Sandra/2019/2019_08_27/"

In [7]:
# rec = Recording("/data/Sandra/2019/2019_08_13/Experiment29.lif")
rec = Recording(folder+"Experiment30.lif")

In [8]:
# rec.parse_metadata()

In [9]:
rec.metadata

,Name,SizeT,SizeX,SizeY,SizeZ,pxSize,pxUnit,bit depth,Frequency,Start time,End time,Duration,line scan
0,Image006,1,1024,1024,1,0.243,µm,uint8,NaN,2019-08-27 09:44:57,NaT,NaT,none
1,Image010,1,1024,1024,1,0.324,µm,uint8,NaN,2019-08-27 09:49:28,NaT,NaT,none
2,Image012,1,1024,1024,1,0.324,µm,uint8,NaN,2019-08-27 09:50:10,NaT,NaT,none
3,Series013,7583,512,512,1,0.648,µm,uint8,1.980,2019-08-27 09:50:40,2019-08-27 10:54:26,0 days 01:03:46.856000900,none
4,Image017,1,1024,1024,1,0.241,µm,uint8,NaN,2048-03-12 10:41:23,NaT,NaT,none
5,Image018,1,1024,1024,1,0.241,µm,uint8,NaN,2019-08-27 11:04:23,NaT,NaT,none
6,Series019,2037,512,512,1,0.482,µm,uint8,1.980,2019-08-27 11:05:13,2019-08-27 11:22:20,0 days 00:17:07.127000808,none
7,Image021,1,1024,1024,1,0.424,µm,uint8,NaN,2019-08-27 11:25:48,NaT,NaT,none
8,Series022,1,1024,1024,1,0.424,µm,uint8,NaN,2048-03-12 11:03:52,NaT,NaT,none
9,Series023,984,512,512,1,0.850,µm,uint8,1.980,2019-08-27 11:26:47,2019-08-27 11:35:02,0 days 00:08:15.813999176,none


In [10]:
nameDict = dict([(name, list(ii)) for ii, name in parse_leica(rec, index=True)])

In [11]:
ser = 'Series035-37'

In [12]:
indices = nameDict[ser]

In [13]:
serNames = rec.metadata.loc[indices,"Name"]

In [14]:
from islets.LineScan import LineScan

In [15]:
for name in serNames:
    rec.import_series(name, isLineScan=True)
    break

In [16]:
linescan = LineScan(
    data = rec.Series[name]["data"][:,0].T,
    metadata = rec.Series[name]["metadata"],
    name = "%s: %s"%(rec.Experiment[:-4], name),
    )

In [17]:
# linescan.detrend(processes=15,fast=False)

In [18]:
linescan.examine()

/opt/tljh/user/envs/physio/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/tljh/user/envs/physio/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/tljh/user/envs/physio/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/tljh/user/envs/physio/lib/python3.7/site-packages/tensorflow/pyth